# Chapter 03: 데이터셋 준비

## 1. 학습 목표

* Fine-tuning에 필요한 데이터셋 유형(Instruction vs Preference)을 이해한다.
* 좋은 데이터와 나쁜 데이터를 명확히 구분할 수 있는 안목을 기른다.
* 중복 제거, 정규화, 품질 평가를 포함한 데이터 클리닝 파이프라인을 구축한다.
* `datasets` 라이브러리를 활용해 데이터를 효율적으로 로드하고 처리한다.

## 2. 데이터 품질의 중요성

### 2.1 핵심 원칙

> **"모델은 데이터만큼만 좋아진다 (Garbage In, Garbage Out)"**

최신 기술, 최고의 하드웨어, 가장 강력한 베이스 모델을 사용하더라도 데이터 품질이 낮으면 결과물은 형편없다. 반면, 작은 모델이라도 고품질 데이터로 학습하면 놀라운 성능을 발휘할 수 있다.

### 2.2 80-20 법칙

Fine-tuning 성능 향상의 **80%는 처음 확보한 5,000개의 고품질 예제**에서 나온다. 무작정 데이터 양을 늘리기보다는 소량이라도 완벽한 데이터를 확보하는 것이 우선이다.

## 3. 데이터셋 유형

Fine-tuning 목적에 따라 크게 두 가지 유형의 데이터셋이 사용된다.

### 3.1 Instruction Dataset (SFT용)

모델에게 "지시를 따르는 법"을 가르치기 위한 데이터다. 질문(Instruction)과 이상적인 답변(Output) 쌍으로 구성된다.

```json
[
    {
        "instruction": "이 에러가 무엇을 의미하는지 설명해줘",
        "input": "TypeError: unsupported operand type(s)",
        "output": "이 에러는 Python에서 서로 호환되지 않는 데이터 타입 간에 연산을 시도할 때 발생한다. 예를 들어 문자열과 정수를 더하려고 할 때 나타날 수 있다."
    }
]

```

### 3.2 Preference Dataset (DPO/RLHF용)

모델을 인간의 가치관에 맞게 정렬(Alignment)하기 위한 데이터다. 동일한 프롬프트에 대해 "선호하는 답변"과 "거부하는 답변" 쌍으로 구성된다.

```json
[
    {
        "prompt": "초보자에게 재귀(Recursion)를 설명해줘",
        "chosen": "재귀를 러시아 인형(마트료시카)에 비유할 수 있다. 인형을 열면 더 작은 인형이 나오고, 가장 작은 인형이 나올 때까지 반복되는 구조와 같다...",
        "rejected": "재귀란 함수가 자기 자신을 호출하는 프로그래밍 기법이다. 기저 조건이 없으면 무한 루프에 빠진다."
    }
]

```

## 4. 좋은 데이터 vs 나쁜 데이터

데이터를 준비할 때 참고할 수 있는 구체적인 기준이다.

In [1]:
# 나쁜 데이터 예시 (지시 불명확, 단답형, 맥락 부재)
bad_example = {
    "instruction": "answer this",
    "input": "what is ML",
    "output": "machine learning"
}

# 좋은 데이터 예시 (명확한 지시, 구조화된 응답, 예시 포함)
good_example = {
    "instruction": "다음 개념에 대해 초보자가 이해하기 쉽게 설명해줘.",
    "input": "머신러닝이란 무엇인가?",
    "output": """머신러닝은 컴퓨터가 명시적으로 프로그래밍되지 않고도 데이터로부터
패턴을 학습하는 인공지능의 한 분야이다.

예를 들어, 스팸 필터는 스팸으로 표시된 이메일을 분석하여 스스로 스팸의 특징을 학습하고 식별한다.

핵심 구성요소:
1. 데이터: 학습할 예제 (교과서 역할)
2. 알고리즘: 학습 방법 (공부법)
3. 모델: 학습된 결과물 (지식)"""
}

print("좋은 데이터의 특징:")
print("- 명확한 지시사항 (Persona, Tone, Format 명시)")
print("- 논리적이고 구조화된 응답")
print("- 구체적인 예시 포함")
print("- 일관된 포맷팅 (Markdown 활용 등)")

좋은 데이터의 특징:
- 명확한 지시사항 (Persona, Tone, Format 명시)
- 논리적이고 구조화된 응답
- 구체적인 예시 포함
- 일관된 포맷팅 (Markdown 활용 등)


## 5. 데이터 클리닝 파이프라인 실습

실무에서 수집한 데이터는 노이즈가 많다. 이를 정제하기 위한 전처리 함수들을 구현한다.

### 5.1 필수 라이브러리 및 유틸리티

In [2]:
import hashlib
import re
import unicodedata

def normalize_text(text):
    """
    텍스트 정규화 함수다.
    유니코드 정규화(NFKC)와 불필요한 공백을 제거한다.
    """
    if not text:
        return ""
    # NFKC 정규화: 호환성 문자를 표준 문자로 변환 (예: ㈜ -> (주))
    text = unicodedata.normalize('NFKC', text)
    # 연속된 공백을 하나의 공백으로 치환
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def remove_duplicates(dataset):
    """
    데이터셋 중복 제거 함수다.
    Instruction과 Input, Output을 합친 내용의 해시값을 비교한다.
    """
    seen_hashes = set()
    unique_data = []

    for item in dataset:
        # 주요 필드를 합쳐서 지문(Fingerprint) 생성
        text = item.get('instruction', '') + item.get('input', '') + item.get('output', '')
        # 정규화 후 해시 생성
        normalized_text = normalize_text(text).lower()
        text_hash = hashlib.md5(normalized_text.encode()).hexdigest()

        if text_hash not in seen_hashes:
            seen_hashes.add(text_hash)
            unique_data.append(item)

    print(f"중복 제거 완료: {len(dataset)} -> {len(unique_data)} (삭제: {len(dataset) - len(unique_data)}개)")
    return unique_data

def calculate_quality_score(item):
    """
    데이터 품질 점수를 계산하는 휴리스틱 함수다.
    점수가 너무 낮은 데이터는 필터링할 수 있다.
    """
    score = 0
    instruction = item.get('instruction', '')
    output = item.get('output', '')

    # 1. 길이 체크 (너무 짧으면 감점)
    if len(output.split()) >= 10: score += 10
    if len(output.split()) >= 50: score += 10

    # 2. 구조적 요소 체크 (개행, 리스트 등)
    if '\n' in output: score += 15

    # 3. 설명적 요소 체크
    explanation_keywords = ['예를 들어', '때문에', '즉', '따라서', '구체적으로']
    if any(kw in output for kw in explanation_keywords): score += 15

    # 4. 입력/출력 중복 체크 (지시사항을 그대로 복사한 경우)
    if instruction.strip() == output.strip():
        score = -100 # 폐기 대상

    return score

In [3]:
# 테스트 데이터
raw_data = [
    good_example,
    good_example, # 중복
    bad_example,
    {"instruction": "hi", "output": "hi"} # 저품질
]

# 파이프라인 실행
print("1. 데이터 정규화 및 중복 제거")
cleaned_data = remove_duplicates(raw_data)

print("\n2. 품질 평가 및 필터링")
final_data = []
for item in cleaned_data:
    score = calculate_quality_score(item)
    print(f"데이터 점수: {score} | 내용: {item['instruction'][:20]}...")
    if score > 0: # 기준점 설정
        final_data.append(item)

print(f"\n최종 데이터 개수: {len(final_data)}")

1. 데이터 정규화 및 중복 제거
중복 제거 완료: 4 -> 3 (삭제: 1개)

2. 품질 평가 및 필터링
데이터 점수: 40 | 내용: 다음 개념에 대해 초보자가 이해하기 ...
데이터 점수: 0 | 내용: answer this...
데이터 점수: -100 | 내용: hi...

최종 데이터 개수: 1


## 6. 데이터 크기 가이드라인

gpt-oss-20b와 같은 모델을 Fine-tuning할 때 목표 성능에 따른 권장 데이터 양이다.

| 데이터 수 (예제 쌍) | 기대 효과 | 비고 |
| --- | --- | --- |
| **500 ~ 1,000** | 기본적인 행동/톤 변화 | 페르소나 주입, 말투 변경에 효과적 |
| **1,000 ~ 5,000** | 견고한 성능 확보 | 새로운 지식 주입보다는 패턴 학습에 적합 |
| **5,000 ~ 10,000** | 우수한 성능 (추천) | 대부분의 상용 SFT 모델 수준 |
| **10,000+** | 도메인 특화 완성 | 새로운 지식 학습 및 복합적 추론 능력 향상 |

## 7. HuggingFace Datasets 활용

실습을 위해 아래 2개의 데이터셋을 로드한다.
1. 영문 공개 데이터셋(`databricks-dolly-15k`)
2. 한글 공개 데이터셋 (`nlpai-lab/databricks-dolly-15k-ko`)

`databricks-dolly-15k` 데이터 셋은 Databricks에서 생성한 오픈소스로, 브레인스토밍, 분류, 비공개 QA, 생성, 정보추출, 공개 QA 및 요약 등의 지침을 포함한 데이터 셋입니다.
Dataset은 Kullm(구름)모델을 개발한 고려대학교 연구소에서 제공하는 databricks-dolly 데이터 셋을 한국어로 번역한 nlpai-lab/databricks-dolly-15k-ko를 사용하도록 하겠습니다.
데이터의 개수는 총 15,011개 입니다. 아래 링크에 가서 라이센스 동의를 클릭해야 로딩할 수 있습니다.

https://huggingface.co/datasets/nlpai-lab/databricks-dolly-15k-ko

### 7.1. Load Dataset: datatricks-dolly-15k

In [5]:
from datasets import load_dataset

# 공개 데이터셋 로드 (Databricks Dolly 15k)
# 실제 학습 시에는 전체 데이터를 사용하지만, 여기서는 예시로 100개만 로드한다.
dataset_name = "databricks/databricks-dolly-15k"
print(f"데이터셋 로드 중: {dataset_name}")

dataset = load_dataset(dataset_name, split="train[:100]")

print(f"\n로드 완료: {len(dataset)}개")
print(f"데이터 구조(Features): {dataset.column_names}")
print("\n[샘플 데이터]")
print(dataset[0])

데이터셋 로드 중: databricks/databricks-dolly-15k

로드 완료: 100개
데이터 구조(Features): ['instruction', 'context', 'response', 'category']

[샘플 데이터]
{'instruction': 'When did Virgin Australia start operating?', 'context': "Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.", 'response': 'Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.', 'category': 'closed_qa'}


### 7.2. Laod Dataset: databricks-dolly-15k-ko

In [6]:
from datasets import load_dataset

# 공개 데이터셋 로드 (Databricks Dolly 15k-ko 한국어 데이터셋)
# 실제 학습 시에는 전체 데이터를 사용하지만, 여기서는 예시로 100개만 로드한다.
dataset_name = "nlpai-lab/databricks-dolly-15k-ko"
print(f"데이터셋 로드 중: {dataset_name}")

dataset = load_dataset(dataset_name, split="train[:100]")

print(f"\n로드 완료: {len(dataset)}개")
print(f"데이터 구조(Features): {dataset.column_names}")
print("\n[샘플 데이터]")
print(dataset[0])

데이터셋 로드 중: nlpai-lab/databricks-dolly-15k-ko


databricks-dolly-15k-ko.jsonl:   0%|          | 0.00/15.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]


로드 완료: 100개
데이터 구조(Features): ['instruction', 'context', 'response', 'category', 'id']

[샘플 데이터]
{'instruction': '버진 오스트레일리아는 언제부터 운영을 시작했나요?', 'context': '버진 오스트레일리아는 버진 오스트레일리아 항공(Virgin Australia Airlines Pty Ltd)의 상호로, 호주에 본사를 둔 항공사입니다. 버진 브랜드를 사용하는 항공사 중 항공기 규모 면에서 가장 큰 항공사입니다. 2000년 8월 31일에 단일 노선에 두 대의 항공기로 버진 블루로 서비스를 시작했습니다.[3] 2001년 9월 호주 앤셋 항공이 파산한 후 호주 국내 시장에서 주요 항공사로 급부상했습니다. 이후 브리즈번, 멜버른, 시드니의 허브를 거점으로 호주 내 32개 도시에 직접 취항하는 항공사로 성장했습니다.[4]', 'response': '버진 오스트레일리아는 2000년 8월 31일에 버진 블루로 서비스를 시작했으며, 단일 노선에 두 대의 항공기를 운항했습니다.', 'category': 'closed_qa', 'id': 0}


## 8. 요약

이번 챕터에서는 성공적인 Fine-tuning을 위한 데이터 준비 과정을 다루었다.

1. **데이터 품질이 최우선이다**: 양보다 질이 중요하며, 80-20 법칙을 기억해야 한다.
2. **명확한 목적에 맞는 유형 선택**: 지시 이행을 위해서는 Instruction Dataset, 인간 가치 정렬을 위해서는 Preference Dataset을 구축한다.
3. **체계적인 파이프라인 구축**: 중복 제거, 정규화, 품질 평가 로직을 통해 노이즈를 최소화해야 한다.

잘 정제된 데이터셋이 준비되었다면, 이제 이를 학습시킬 하드웨어와 소프트웨어 환경을 갖춰야 한다.

다음 챕터는 **Chapter 04: 환경 구축**으로, GPU 설정부터 필수 라이브러리 설치까지 실습 환경을 완벽하게 세팅하는 방법을 다룬다.